In [13]:
#import library
import pandas as pd
import numpy as np
import nltk
import string
import csv
import re
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import warnings
warnings.filterwarnings("ignore")

In [14]:
#oop text preprocessing
class process:
    def __init__(self, file):
        self.file = pd.read_csv(file)
        
    def baca(self):
        komena = self.file
        return komena
    
    def simpan(self):
        self.file.to_excel('data_2_inggris.xlsx')
        
    def case_folding(self):
        self.file['Comment']= self.file['Comment'].str.lower()
        
    def remove_number(self, text):
        def remove_number(text):
            return re.sub(r"\d+", "", text)
        self.file['Comment'] = self.file['Comment'].apply(remove_number)
    
    def remove_punemot(self, text): #punctuation & emoticon
        self.file['Comment']= self.file['Comment'].str.replace('[^\w\s]','')
        
    def remove_url(self, text):
        def remove_URL(sample):
            return re.sub(r"http\S+", "", sample)
        self.file['Comment'] = self.file['Comment'].apply(remove_URL)
    
    def stopword(self, text):
        stopword = nltk.corpus.stopwords.words('indonesian')
        
        stopword.extend(['dehhh','wkwk', 'busukin','tersesat', 'mampus', 'aura', 'kesel','ampun','lu','loe','dehhh','la',
                         'eh','awokawoaok', 'cu', 'hehehehe','lah', 'yap','paan', 'cui', 'cak','yuuuk','coki'])
        
        txt_stopword = pd.read_csv('stopwords-id.txt', names= ['stopwords'], header= None)

        stopword.extend(txt_stopword['stopwords'][0].split(' '))

        stopword = set(stopword)

        def stopwords(text):
            text=  [word for word in text if word not in stopword]
            return text
        
        self.file['Comment']= self.file['Comment'].apply(stopwords)
        
    def token(self, text):
        def token(text):
            text=re.split('\W+',text)
            return text
        self.file['Comment'] = self.file['Comment'].apply(token)
    
    def normalisasi(self, text):
        normalized_word = pd.read_csv('normalisasi.csv')

        normalized_word_dict={}

        for index, row in normalized_word.iterrows():
            if row[0] not in normalized_word_dict:
                normalized_word_dict[row[0]] = row[1]
    
        def normalized_term(document):
            return [normalized_word_dict[term] if term in normalized_word_dict else term for term in document]

        self.file['Comment'] = self.file['Comment'].apply(normalized_term)
        
    def stemd(self,text):
        factory = StemmerFactory()
        stemmer = factory.create_stemmer()
        
        def stemmed_wrapper(term):
            return stemmer.stem(term)
        term_dict= {}

        for document in self.file['Comment']:
            for term in document:
                if term not in term_dict:
                    term_dict[term] = ' '

        for term in term_dict:
            term_dict[term] = stemmed_wrapper(term)

        def get_stemmed_term(document):
            return [term_dict[term] for term in document]

        self.file['Comment'] = self.file['Comment'].apply(get_stemmed_term)
        
    def fitt_stopword(self,text):
        def fit_stopwords(text):
            text= np.array(text)
            text= ' '.join(text)
            return(text)

        self.file['Comment']= self.file['Comment'].apply(lambda x: fit_stopwords(x))

In [15]:
dataset = process('data_2.csv')

In [16]:
cf= dataset.case_folding()

In [17]:
clean1 = dataset.remove_number(cf)

In [18]:
clean2 = dataset.remove_punemot(clean1)

In [19]:
clean3 = dataset.remove_url(clean2)

In [20]:
token = dataset.token(clean3)

In [21]:
normal = dataset.normalisasi(token)

In [22]:
sw = dataset.stopword(normal)

In [23]:
stem = dataset.stemd(sw)

In [24]:
fit = dataset.fitt_stopword(stem)

In [25]:
baca= dataset.baca()
print(baca)

      Unnamed: 0                                            Comment
0              0                                          bneran ps
1              1                             mati indonesia duka ya
2              2       kena batu jelekquotin umat muslim pede pakai
3              3                  gayyy kau nonton bokep cowo pulak
4              4  sempet nya muka ngelawak pas tangkap makan tuh...
...          ...                                                ...
3149        3149                                      sayang cokiii
3150        3150                                                   
3151        3151                                        first nichh
3152        3152                               cokii emng totalitas
3153        3153                                           pertamaa

[3154 rows x 2 columns]


In [ ]:
simpan1 = dataset.simpan()

In [26]:
from textblob import TextBlob
import pandas as pd

#oop klasifikasi
class klasifikasi():
    
    def __init__(self, file):
        self.file = pd.read_csv(file)
        self.file['Text_English'] = self.file['Text_English'].astype(str)
        
    def baca(self):
        komena = self.file
        return komena
    
    def simpan(self):
        self.file.to_csv('data_2_inggris_class.csv')
    
    def textblob(self):
        for yt in self.file.Text_English:
            clean = yt
    
            blob_object = TextBlob (clean)
            hasil = blob_object.tags
            print(hasil)
            
    def polarity(self, text):
        polarityy = lambda x: TextBlob(x).sentiment.polarity

        self.file['Polarity'] = self.file['Text_English'].apply(polarityy)
        
    def analysis(self, text):
        def analysis(score):
            if score > 0 :
                return 'positive'
            elif score == 0:
                return 'neutral'
            else:
                return 'negative'
        self.file['Score']= self.file['Polarity'].apply(analysis)
    
    def hitung(self, text):
        print('netral :', (sum(self.file['Score']=='neutral')))
        print('positif :', (sum(self.file['Score']=='positive')))
        print('negatif :', (sum(self.file['Score']=='negative')))

In [27]:
dataset2 = klasifikasi('data_2_inggris.csv')

In [28]:
blob = dataset2.textblob()

[('Bernan', 'NNP'), ('PS', 'NNP')]
[('die', 'NN'), ('indonesia', 'NN'), ('sorrow', 'NN')]
[('hit', 'VBN'), ('by', 'IN'), ('jelekquotin', 'NN'), ('stones', 'NNS'), ('with', 'IN'), ('Muslim', 'NNP'), ('people', 'NNS'), ('wearing', 'VBG')]
[('Gayyy', 'NNP'), ('You', 'PRP'), ('Watch', 'VBP'), ('Pokep', 'NNP'), ('Cowo', 'NNP'), ('Pulak', 'NNP')]
[('Sempet', 'NN'), ('is', 'VBZ'), ('the', 'DT'), ('face', 'NN'), ('of', 'IN'), ('singing', 'VBG'), ('when', 'WRB'), ('catching', 'VBG'), ('a', 'DT'), ('real', 'JJ'), ('dark', 'NN'), ('jokes', 'NNS')]
[('Raw', 'NNP'), ('is', 'VBZ'), ('shocked', 'VBN'), ('Layb', 'NNP')]
[('The', 'DT'), ('contents', 'NNS'), ('of', 'IN'), ('the', 'DT'), ('comments', 'NNS'), ('differently', 'RB'), ('appeal', 'VBP'), ('Jefri', 'NNP'), ('Nicol', 'NNP'), ('when', 'WRB'), ('they', 'PRP'), ('got', 'VBD'), ('sad', 'JJ'), ('drugs', 'NNS')]
[('SANGKA', 'NNP'), ('DAH', 'NNP'), ('CORPURATIVE', 'NNP'), ('PLAGAS', 'NNP'), ('BETIONS', 'NNP'), ('Seba', 'NNP'), ('People', 'NNP'), ('Lik

In [29]:
polar = dataset2.polarity(blob)

In [30]:
analysis = dataset2.analysis(polar)

In [31]:
htng = dataset2.hitung(analysis)

netral : 1997
positif : 735
negatif : 344


In [32]:
baca= dataset2.baca()
print(baca)

      Unnamed: 0                                            Comment  \
0              0                                          bneran ps   
1              1                             mati indonesia duka ya   
2              2       kena batu jelekquotin umat muslim pede pakai   
3              3                  gayyy kau nonton bokep cowo pulak   
4              4  sempet nya muka ngelawak pas tangkap makan tuh...   
...          ...                                                ...   
3071        3148                              cokcok gw sedihh liat   
3072        3149                                      sayang cokiii   
3073        3151                                        first nichh   
3074        3152                               cokii emng totalitas   
3075        3153                                           pertamaa   

                                           Text_English  Polarity     Score  
0                                            Bernan PS.    0.0000   n

In [ ]:
simpan2 = dataset2.simpan()

In [33]:
import pandas as pd
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score

In [34]:
#oop analisi svm
class analisis:
    def __init__(self, file):
        self.file = pd.read_csv(file)
        
    def baca(self):
        komena = self.file
        return komena
    
    def wc(self):
        text = ' '.join(yt for yt in self.file['Comment'])

        WordCloud = WordCloud(width = 600, height = 300, random_state = 21, max_font_size = 200, collocations = False).generate(text)
        plt.figure(figsize = (20,10))
        plt.imshow(WordCloud, Interpolation = 'bilinear')
        plt.axis('off')
    
    def createdatatrain(self):
        self.file['Text_English']= self.file['Text_English'].astype(str)
        y= self.file['Score'].values
        x= self.file['Text_English'].values
        self.file.x_train, self.file.x_test, self.file.y_train, self.file.y_test = train_test_split(x, y, stratify=y, random_state=1, test_size=0.2, shuffle= True)
        print(self.file.x_train.shape)
        print(self.file.x_test.shape)
        
    def hitung(self,text):
        vectorizer = CountVectorizer(binary= True, stop_words ='english')
        vectorizer.fit (list(self.file.x_train)+ list(self.file.x_test))
        self.file.x_train_vec = vectorizer.transform(self.file.x_train)
        self.file.x_test_vec = vectorizer.transform(self.file.x_test)
        print(self.file.x_train_vec.shape)
        print(self.file.x_test_vec.shape)
    
    def linear(self,text):
        svm_linear= svm.SVC(kernel='linear', probability= True, C=100)
        prob= svm_linear.fit(self.file.x_train_vec, self.file.y_train).predict_proba(self.file.x_test_vec)

        y_pred_svm_linear= svm_linear.predict(self.file.x_test_vec)
        
        print("Accuracy Linear :", accuracy_score(self.file.y_test, y_pred_svm_linear)*100,'%')
        scores = cross_val_score(svm_linear, self.file.x_train_vec, self.file.y_train, cv=10)

        print('\n')
        print('confusion matrix linear :')
        print(confusion_matrix(self.file.y_test, y_pred_svm_linear))

        print('\n')
        print('precision linear :', precision_score(self.file.y_test, y_pred_svm_linear, average='macro'))

        print('\n')
        print('recall linear :', recall_score(self.file.y_test, y_pred_svm_linear, average='macro'))

        print('\n')
        print('cross validation linear :', scores)
        
    def rbf(self,text):
        rbf= svm.SVC(kernel='rbf', probability= True, C=100, gamma= 0.01 )
        prob= rbf.fit(self.file.x_train_vec, self.file.y_train).predict_proba(self.file.x_test_vec)

        y_pred_svm_rbf= rbf.predict(self.file.x_test_vec)
        
        print("Accuracy RBF :", accuracy_score(self.file.y_test, y_pred_svm_rbf)*100,'%')
        scores = cross_val_score(rbf, self.file.x_train_vec, self.file.y_train, cv=10)

        print('\n')
        print('confusion matrix rbf :')
        print(confusion_matrix(self.file.y_test, y_pred_svm_rbf))

        print('\n')
        print('precision rbf :', precision_score(self.file.y_test, y_pred_svm_rbf, average='macro'))

        print('\n')
        print('recall rbf :', recall_score(self.file.y_test, y_pred_svm_rbf, average='macro'))

        print('\n')
        print('cross validation rbf :', scores)

In [35]:
data = analisis('data_2_inggris_class.csv')

In [36]:
train = data.createdatatrain()

(2460,)
(616,)


In [37]:
htg = data.hitung(train)

(2460, 4582)
(616, 4582)


In [38]:
linear = data.linear(htg)

Accuracy Linear : 91.07142857142857 %


confusion matrix linear :
[[ 52  16   1]
 [  2 392   6]
 [  8  22 117]]


precision linear : 0.8979619904976245


recall linear : 0.8431805185842453


cross validation linear : [0.93902439 0.88211382 0.87804878 0.8902439  0.88211382 0.89430894
 0.88617886 0.86585366 0.88211382 0.89837398]


In [39]:
rbf = data.rbf(htg)

Accuracy RBF : 93.18181818181817 %


confusion matrix rbf :
[[ 52  14   3]
 [  2 394   4]
 [  3  16 128]]


precision rbf : 0.9298913776404674


recall rbf : 0.8697904959085084


cross validation rbf : [0.93902439 0.87398374 0.8902439  0.8902439  0.8902439  0.92276423
 0.91056911 0.8902439  0.89430894 0.92276423]
